In [2]:
import os
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv
load_dotenv()

# 환경변수에서 접속 정보 읽기
db_user = os.getenv("POSTGRES_USER")
db_pwd = os.getenv("POSTGRES_PASSWORD")
db_host = os.getenv("POSTGRES_HOST")
db_port = os.getenv("POSTGRES_PORT")
db_name = os.getenv("POSTGRES_DB")

if not all([db_user, db_pwd, db_host, db_port, db_name]):
    raise ValueError(f"필요한 DB 환경변수{db_user}, {db_pwd}, {db_host}, {db_port}, {db_name}가 설정되지 않았습니다.")

DATABASE_URL = f"postgresql+psycopg2://{db_user}:{db_pwd}@{db_host}:{db_port}/{db_name}"



In [ ]:
# 디비에서 테이블 데이터를 읽어와 df에 할당
QUERY = """
SELECT *
FROM t3_chunks
"""
engine = create_engine(DATABASE_URL)
df = pd.read_sql(QUERY, con=engine)

In [ ]:
print(f"총 행 수: {len(df)}")
display(df.head(5))

총 행 수: 116418


,id,c_id,chunk_text,embedding
0,341,guide_kr_2023_1184_1,치료의 장단점을 환자와 보호자에게 충분히 설명하고 동의를 구하는 것이 중요합니다. ...,"[0.037429124,0.013210279,0.006558094,0.0789605..."
1,342,guide_kr_2023_1184_1,"이를 위해 근치적 수술, 항암화학요법, 방사선치료 등이 병행되며, 최신 치료법과 기...","[0.011459817,0.015995558,0.014979469,0.0540936..."
2,343,guide_kr_2023_1184_2,"항암화학요법은 간 독성을 유발할 수 있으며, 전이 병소가 완전 관해 상태에 도달하면...","[-0.027085433,0.025721556,-0.0023195504,0.0421..."
3,344,guide_kr_2023_1184_2,간 전이 치료에서 고주파 소작술(RFA)은 수술적 치료에 비해 재발률과 5년 생존율...,"[-0.003236592,0.028439892,0.03457713,0.0523871..."
4,345,guide_kr_2023_1184_2,"전이성 또는 재발성 대장암에서 절제가 불가능한 경우라도, 장 폐색이나 출혈 등 증상...","[0.010366654,0.041587394,0.042875677,0.0199180..."


In [ ]:
import sys
from pathlib import Path

project_root = Path(r"C:\dev\ai-camp\SKN18-4th-4team")
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

import os
import pandas as pd
from IPython.display import display
import retriever
from importlib import reload


# 서비스 코드가 바뀌었으면 리로드
reload(retriever)

# 벡터 리트리버 인스턴스 가져오기
vector_retriever = retriever.get_vector_retriever()

# 검색할 텍스트
query_text = "검사 결과 해석 방법"

# top_k, 최소 유사도 등 필요에 따라 지정
docs = vector_retriever.search(query=query_text, top_k=5)

print(f"검색 결과 {len(docs)}건")
results = []
for doc in docs:
    meta = doc.metadata or {}
    results.append(
        {
            "id": meta.get("id"),
            "similarity": meta.get("similarity"),
            "distance": meta.get("distance"),
            "excerpt": doc.page_content[:120] + ("..." if len(doc.page_content) > 120 else ""),
        }
    )

display(pd.DataFrame(results))

검색 결과 5건


,chunk_id,c_id,similarity,distance,excerpt
0,71749,KJM_2024_34683_1,0.665872,0.334128,검사 결과는 반드시 환자의 병력을 바탕으로 해석해야 합니다.
1,69723,KJM_2021_34570_1,0.633592,0.366408,"검사 결과를 해석할 때는 검사법의 특성과 한계를 충분히 이해하고, 환자의 임상 양상..."
2,102713,KSEM_2006_36735_1,0.614863,0.385137,"검사 시행 시 보호자의 동의를 구했으며, 동의를 얻지 못한 경우 의사의 판단에 따라..."
3,55937,KJM_2011_33705_1,0.595163,0.404837,검사 결과 의미 있는 협착 병변은 관찰되지 않았습니다. 검사 후 환자는 병실로 이동...
4,58120,KJM_2012_33850_1,0.566526,0.433474,"검사 결과 백혈구 수치가 11,670/mm³로 증가하였고, CRP는 12 mg/dL..."


In [ ]:
def build_context_from_results(docs):
    """
    검색 결과(Document 리스트)를 LLM 컨텍스트 텍스트로 변환합니다.
    
    Args:
        docs: 검색된 Document 리스트 (retriever.search() 결과)
        
    Returns:
        str: 포맷된 컨텍스트 텍스트
    """
    context_parts = []
    for idx, doc in enumerate(docs, 1):
        meta = doc.metadata or {}
        doc_id = meta.get("id", "N/A")
        similarity = meta.get("similarity")
        distance = meta.get("distance")
        content = doc.page_content
        
        # 거리 및 유사도 정보 포맷팅
        distance_str = f"{distance:.4f}" if distance is not None else "N/A"
        similarity_str = f"{similarity:.4f}" if similarity is not None else "N/A"
        
        context_parts.append(
            f"[문서 {idx}]\n"
            f"id: {doc_id}\n"
            f"유사도: {similarity_str}\n"
            f"거리: {distance_str}\n"
            f"내용:\n{content}\n"
        )
    
    return "\n\n".join(context_parts)


In [11]:
def answer_with_rag(query: str, top_k: int = 10) -> str:
    """
    RAG(Retrieval-Augmented Generation)를 사용하여 질문에 답변합니다.
    
    1) DB에서 코사인유사도 TOP_K 검색
    2) 그 결과를 컨텍스트로 묶어서
    3) OpenAI Chat 모델에 던져서 최종 답변 생성
    
    Args:
        query: 사용자 질문
        top_k: 검색할 문서 개수 (기본값: 10)
        
    Returns:
        str: LLM이 생성한 답변
    """
    import os
    from openai import OpenAI
    from dotenv import load_dotenv
    
    load_dotenv()
    
    # 환경변수에서 설정 불러오기
    openai_api_key = os.getenv("OPENAI_API_KEY")
    openai_base_url = os.getenv("OPENAI_BASE_URL")
    chat_model = os.getenv("OPENAI_CHAT_MODEL", "gpt-4o-mini")
    
    if not openai_api_key:
        raise ValueError("OPENAI_API_KEY 환경변수가 설정되지 않았습니다.")
    
    # 1) DB에서 검색
    vector_retriever = retriever.get_vector_retriever()
    docs = vector_retriever.search(query=query, top_k=top_k)
    
    if not docs:
        return "관련 문서를 찾을 수 없습니다."
    
    # 2) LLM 컨텍스트 만들기
    context_text = build_context_from_results(docs)
    
    # (선택) 확인용 출력
    print("\n===== [검색된 상위 문서들] =====\n")
    print(context_text[:2000])  # 너무 길면 앞부분만
    
    # 3) LLM 호출
    client = OpenAI(
        api_key=openai_api_key,
        base_url=openai_base_url if openai_base_url else None
    )
    
    messages = [
        {
            "role": "system",
            "content": (
                "너는 의학/진료지침 관련 질문에 답하는 한국어 전문가야. "
                "아래 제공하는 '참고 문서들' 내용만 기반으로 답변해야 하고, "
                "참고문서을 종합해서 논리적으로 답변해."
            ),
        },
        {
            "role": "user",
            "content": (
                "아래는 데이터베이스에서 가져온 관련 문서 조각들이야.\n\n"
                "===== 참고 문서 시작 =====\n"
                f"{context_text}\n"
                "===== 참고 문서 끝 =====\n\n"
                f"위 문서들을 참고해서 다음 질문에 한국어로 친절하게 답변해줘:\n\n"
                f"질문: {query}"
            ),
        },
    ]
    
    resp = client.chat.completions.create(
        model=chat_model,
        messages=messages,
        temperature=0.3,   # 너무 헛소리 안 하게 살짝 낮게
    )
    
    answer = resp.choices[0].message.content
    return answer

In [12]:
if __name__ == "__main__":
    q = "진료지침이 뭔지 알려줘"

    final_answer = answer_with_rag(q, top_k=5)

    print("\n===== [최종 답변] =====\n")
    print(final_answer)


===== [검색된 상위 문서들] =====

[문서 1]
c_id: guide_kr_2023_1182_1
유사도(거리): 0.4191
내용:
진료지침은 과학적 근거를 바탕으로 의사들에게 효과적인 치료 방법을 제시하고, 위험한 치료를 경고하며, 치료 대안의 장단점을 요약하여 최신 정보를 제공합니다. 이를 통해 진료의 일관성을 높이고, 의사와 환자 모두에게 자신감을 제공합니다.


[문서 2]
c_id: guide_kr_2023_1293_169
유사도(거리): 0.4269
내용:
진료지침의 목적, 대상, 예상되는 편익 또는 결과는 구체적으로 서술되어야 하며, 이를 주로 활용할 사용자 집단도 명확히 규정해야 합니다. 예를 들어, 류마티스내과 전문의, 가정의, 요통 환자 등과 같은 구체적인 사용자 정보를 포함해야 합니다.


[문서 3]
c_id: guide_kr_2023_1182_1
유사도(거리): 0.4301
내용:
진료지침은 의사와 환자 간의 의사결정을 돕기 위해 체계적으로 개발된 도구로, 적절한 진료 내용을 기술하고 있습니다. 이는 효과가 입증된 치료를 증진하고, 효과가 없는 치료를 지양하여 건강 결과를 향상시키는 것을 목표로 합니다.


[문서 4]
c_id: guide_kr_2023_1293_352
유사도(거리): 0.4523
내용:
진료지침은 일정한 개정 주기와 방법론을 제시하며, 개정을 결정하는 판단 기준이 명시됩니다.


[문서 5]
c_id: KJG_2010_31237_1
유사도(거리): 0.4548
내용:
진료지침은 의료의 질 향상을 목표로 하며, 이를 효과적으로 확산하고 실행하기 위한 노력이 동반되어야 합니다.


===== [최종 답변] =====

진료지침은 과학적 근거를 바탕으로 의사들에게 효과적인 치료 방법을 제시하고, 위험한 치료를 경고하며, 치료 대안의 장단점을 요약하여 최신 정보를 제공하는 도구입니다. 이러한 지침은 의사와 환자 간의 의사결정을 돕기 위해 체계적으로 개발되며, 효과가 입증된 치료를 증진하고 효과가 없는 치료를 지양

In [13]:
if __name__ == "__main__":
    q = "38세 여자가 6개월 전부터 전신 통증을 호소하며 내원했다. 환자는 통증의 정확한 부위를 특정하기 어렵다고 하며, 팔다리와 몸통에 걸쳐 전반적인 통증이 있어 일상생활에 지장이 있다고 한다. 또한 쉽게 피로해지고 밤에 잠을 잘 이루지 못한다고 한다. 신체 진찰 결과, 관절의 부종, 열감, 근력 이상은 없었으며, 수동 및 능동 관절 검사와 혈액 검사 모두 정상 소견이었다. 이 환자의 진단으로 가장 적절한 것은?  \n1) 섬유근육통  \n2) 다발근육염  \n3) 근막통증증후군  \n4) 류마티스 다발성 근통  \n5) 복합부위통증증후군"

    final_answer = answer_with_rag(q, top_k=10)

    print("\n===== [최종 답변] =====\n")
    print(final_answer)


===== [검색된 상위 문서들] =====

[문서 1]
c_id: KJM_2014_34058_1
유사도(거리): 0.3183
내용:
28세 여성 환자는 4개월간 지속된 피로감, 체중 감소(7kg), 발열 증상으로 외부 병원을 방문하였으며, 말초혈액도말검사 및 골수검사에서 만성질병성 빈혈 외에 특이 소견은 없었습니다. 대증치료에도 빈혈이 호전되지 않아 흉부 및 복부 전산화단층촬영(CT)을 시행한 결과, 위에 종양이 관찰되어 본원으로 전원되었습니다.


[문서 2]
c_id: KJG_2008_3995_1
유사도(거리): 0.3266
내용:
48세 여성 환자가 하복부 통증, 설사, 발열을 주소로 병원에 내원하였습니다. 환자는 4년 전부터 반복적인 복통과 혈변 증상이 있었으며, 외부 병원에서 상부위장관 및 대장내시경 검사를 받았으나 특이 소견은 발견되지 않았습니다.


[문서 3]
c_id: KJM_2018_34385_1
유사도(거리): 0.3372
내용:
36세 여성 환자가 좌측 무릎 관절의 종창과 통증을 주소로 내원하였습니다. 환자는 내원 3개월 전부터 비루 증상이 있었으며, 혈액 검사에서 혈청 총 면역글로불린 E의 증가가 확인되었습니다.


[문서 4]
c_id: KJM_2018_34396_1
유사도(거리): 0.3379
내용:
28세 여성 환자는 1개월 전부터 연하통증을 호소하며 내원하였습니다. 환자는 연하통증 외에도 구역, 구토, 식욕 감소, 그리고 야간 발한 증상을 보였습니다.


[문서 5]
c_id: KJM_2013_33957_1
유사도(거리): 0.3431
내용:
48세 여성 환자는 두통, 피로감, 오심, 희발 월경, 다뇨 증상을 호소하며 내원하였습니다. 과거력 및 가족력에서 특이소견은 없었으며, 3년 전부터 희발 월경 상태였으나 추가적인 검진은 받지 않았습니다.


[문서 6]
c_id: KJM_2006_32562_1
유사도(거리): 0.3441
내용:
73세 여성 환자가 옻 원액 5g을 두 차례 복용한 후 심와부 통증, 오심, 구토, 토혈, 

In [14]:
if __name__ == "__main__":
    q = "비 ST 분절 상승 심근경색의 증상에 대해 알려줘"

    final_answer = answer_with_rag(q, top_k=10)

    print("\n===== [최종 답변] =====\n")
    print(final_answer)


===== [검색된 상위 문서들] =====

[문서 1]
c_id: KSEM_2008_34865_1
유사도(거리): 0.3213
내용:
그러나 ST 분절 상승이 항상 심근경색을 의미하는 것은 아닙니다. ST 분절 상승의 원인으로는 급성 심근경색 외에도 좌심실비대, 좌각차단, 양성 조기재분극, 심실류 등이 있습니다.


[문서 2]
c_id: KSEM_2000_36183_1
유사도(거리): 0.3705
내용:
심전도에서 ST 분절 상승은 급성 심근경색증을 시사하는 중요한 진단적 단서가 될 수 있지만, ST 분절 상승 자체만으로는 특이성을 가지지 못합니다. 이는 급성 심근경색증 외에도 다양한 원인에 의해 ST 분절 상승이 나타날 수 있기 때문입니다.


[문서 3]
c_id: KSEM_2018_35765_1
유사도(거리): 0.3716
내용:
급성심근경색을 암시하는 증상은 가슴, 상지, 턱, 명치 부위의 불편감이 20분 이상 지속되는 전형적인 증상에서부터 호흡곤란, 피로감, 두근거림 등 다양한 형태로 나타날 수 있습니다. 최근에는 ST분절 상승 급성심근경색(STEMI)보다 비ST분절 상승 급성심근경색(NSTEMI)의 발생이 증가하는 추세를 보이고 있습니다.


[문서 4]
c_id: KSEM_2008_34865_1
유사도(거리): 0.3781
내용:
그러나 이러한 소견은 전벽 ST 분절 상승 심근경색에서도 나타날 수 있어 감별이 어렵습니다. 특히, ST 분절의 모양이 오목한 경우, 전벽 ST 분절 상승 심근경색 환자의 약 43%에서 양성 조기 재분극과 유사한 소견을 보인다고 보고되었습니다.


[문서 5]
c_id: KSEM_2000_36183_1
유사도(거리): 0.3799
내용:
BER의 경우, ST 분절 상승의 첫 부분이 함몰된 특징이 있으며, 이는 급성 심근경색증과의 감별점이 됩니다. 좌심실비대는 비심근경색증에서 ST 분절 상승의 가장 흔한 원인으로 알려져 있으며, 약 70%에서 ST 분절 또는 T파의 변화가 나타납니다.


[문서 6]
c_id: KS

In [15]:
if __name__ == "__main__":
    q = "섬유근육통에 대해 알려줘"

    final_answer = answer_with_rag(q, top_k=10)

    print("\n===== [최종 답변] =====\n")
    print(final_answer)


===== [검색된 상위 문서들] =====

[문서 1]
c_id: KJM_2006_32957_1
유사도(거리): 0.4241
내용:
섬유근육통은 중추신경계의 이상이 원인으로, 항우울제와 항전간제가 흔히 사용됩니다. 척추통증은 원인이 다양하며, 수술이 필요한 경우와 이차적 원인에 의한 통증을 감별하는 것이 중요합니다.


[문서 2]
c_id: KSEM_2002_36333_1
유사도(거리): 0.4618
내용:
섬유근육통은 주로 여성에게 많이 발생하지만, MPS는 남녀 모두에서 비슷한 비율로 발생합니다. 본 연구에서는 남성 환자가 여성 환자보다 2배 많았는데, 이는 연구가 시행된 지역의 남성 근로자 비율이 높았기 때문으로 보입니다.


[문서 3]
c_id: KJM_2009_33486_1
유사도(거리): 0.4758
내용:
섬유근통의 발병 기전으로는 중추신경계에서 통증 조절에 문제가 생긴다는 가설이 가장 널리 받아들여지고 있습니다. 이외에도 근육 자체의 문제를 제기하는 골격근 가설이나, 비정상적인 수면 패턴과 관련된 수면장애 가설 등이 있습니다.


[문서 4]
c_id: KJM_2009_33486_1
유사도(거리): 0.5121
내용:
섬유근통은 염증성 질환이 아니기 때문에 관절 변형이나 불구로 이어지지 않으며, 정신 질환과도 구별됩니다. 환자들에게 이러한 점을 명확히 설명하고, 잘못된 치료법에 의존하지 않도록 하는 것이 중요합니다.


[문서 5]
c_id: KJM_2006_32959_1
유사도(거리): 0.5382
내용:
섬유질 섭취와 물 섭취는 대부분의 경우 도움이 되지만, 일부에서는 장내 가스를 증가시켜 변비를 악화시킬 수 있으므로 주의가 필요합니다. 또한, 균형 잡힌 식사를 위해 영양사와의 상담이 필수적입니다.


[문서 6]
c_id: KJM_2024_34676_1
유사도(거리): 0.5646
내용:
섬유화가 진행되면 망상형 음영, 견인성 기관지확장증, 벌집 모양 변화 등이 나타날 수 있습니다. 병리 소견에서는 비섬유성 과민성폐렴의 경우 균일한

In [16]:
if __name__ == "__main__":
    q = "적혈구형성인자 증상인 환자에 대한 사례를 알려줘"

    final_answer = answer_with_rag(q, top_k=10)

    print("\n===== [최종 답변] =====\n")
    print(final_answer)


===== [검색된 상위 문서들] =====

[문서 1]
c_id: KJG_2010_31307_1
유사도(거리): 0.4499
내용:
위험인자로는 고령, 남성, 당뇨, 고혈압, 심혈관 질환, 음주, 위출구 폐색, 신부전, 악성 종양, 저산소증, 과응고 상태 등이 있습니다. 본 증례는 48세 남성 환자로, 토혈과 삼킴곤란을 주소로 내원하였습니다.


[문서 2]
c_id: KSEM_2000_36185_1
유사도(거리): 0.4597
내용:
응급실에서 흔히 접하는 흉통, 호흡곤란, 현훈 등의 증상은 심인성 여부를 판단하는 것이 매우 중요합니다. 이는 환자의 검사, 처치, 협진 의뢰에 있어 필수적인 과정입니다.


[문서 3]
c_id: guide_kr_2023_1478_1
유사도(거리): 0.4604
내용:
적혈구 생성인자 치료는 수혈을 줄이는 데 효과적일 수 있으나, 부작용과 비용을 고려해야 합니다. 적혈구 수혈 시 저장 기간은 환자의 안전에 중요한 요소입니다.


[문서 4]
c_id: KJM_2007_33083_1
유사도(거리): 0.4607
내용:
객관적인 증상을 규명하기 위해 적절한 검사를 시행하고, 환자 개개인에 맞는 치료를 제공해야 합니다.


[문서 5]
c_id: KJM_2014_34089_1
유사도(거리): 0.4756
내용:
고혈압 환자와 고혈압 위험군에서 중등도의 체중 감소는 혈압을 유의미하게 낮추었으며, 체중이 5~10% 줄어들면 중성지방이 현저히 감소하고 HDL은 증가하는 효과가 있었습니다. 본 연구는 당뇨병 발생 고위험군을 대상으로 20세 때와 현재의 체중 변화를 분석하고, 체질량지수의 변화 양상과 체중 변화량에 따른 인슐린저항성 및 대사증후군 지표와의 연관성을 조사하였습니다.


[문서 6]
c_id: KJM_2013_33890_1
유사도(거리): 0.4768
내용:
만성 안정성 심부전 환자에게는 경구 이뇨제를 사용합니다. 이뇨제로 증상이 호전되지 않을 경우, 혈관 확장제나 심근수축력을 증가시키는 약물(디곡신, 카테콜아민 계열 강심